## NYC Flights Database

In [1]:
import datetime
import numpy as np
import pandas as pd
import plotly.graph_objs as go
import plotly.plotly as py

from ipywidgets import widgets 
from IPython.display import display, clear_output, Image
from plotly.graph_objs import *
from plotly.widgets import GraphWidget

<IPython.core.display.Javascript object>

We'll be making an application to take a look at delays from all flights out of NYC in the year 2013.

In [2]:
df = pd.read_csv('nycflights.csv')
df = df.drop(df.columns[[0]], axis=1) 

In [3]:
df.head()

,year,month,day,dep_time,dep_delay,arr_time,arr_delay,carrier,tailnum,flight,origin,dest,air_time,distance,hour,minute
0,2013,1,1,517.0,2.0,830.0,11.0,UA,N14228,1545,EWR,IAH,227.0,1400,5.0,17.0
1,2013,1,1,533.0,4.0,850.0,20.0,UA,N24211,1714,LGA,IAH,227.0,1416,5.0,33.0
2,2013,1,1,542.0,2.0,923.0,33.0,AA,N619AA,1141,JFK,MIA,160.0,1089,5.0,42.0
3,2013,1,1,544.0,-1.0,1004.0,-18.0,B6,N804JB,725,JFK,BQN,183.0,1576,5.0,44.0
4,2013,1,1,554.0,-6.0,812.0,-25.0,DL,N668DN,461,LGA,ATL,116.0,762,5.0,54.0


Let's get the set of all the `airlines`, so that we can type the right things into the search box later. 

In [4]:
df['carrier'].unique()

array(['UA', 'AA', 'B6', 'DL', 'EV', 'MQ', 'US', 'WN', 'VX', 'FL', 'AS',
       '9E', 'F9', 'HA', 'YV', 'OO'], dtype=object)

Let's assign the widgets that we're going to be using in our app. In general all these widgets will be used to filter the data set, and thus what we visualize.

In [5]:
month = widgets.FloatSlider(
    value=1.0,
    min=1.0,
    max=12.0,
    step=1.0,
    description='Month:',
    continuous_update = False
)

use_date = widgets.Checkbox(
    description='Date: ',
    value=True,
)

container = widgets.HBox(children=[use_date, month] )

textbox = widgets.Text(
    description='Airline:   ',
    value='Delta',
)

origin = widgets.Dropdown(
    options=list(df['origin'].unique()),
    value='LGA',
    description='Origin Airport:',
)


### Assign an emptry graph widget with two traces
g = GraphWidget('https://plot.ly/~kevintest/1192/')

Let now write a function that will handle the input from the widgets, and alter the state of the graph.

In [6]:
def validate():
    # print "validating"
    if origin.value in df['origin'].unique() and textbox.value in df['carrier'].unique():
        return True
    else:
        return False

def response(change):
    if validate():
        # print "using date"
        if use_date.value:
            filter_list = [i and j and k for i,j,k in 
                           zip(df['month'] == month.value, df['carrier'] == textbox.value, 
                               df['origin'] == origin.value)]
            temp_df = df[filter_list]
            x1 = temp_df['arr_delay']
            x2 = temp_df['dep_delay']
            g.restyle({'x': [x1], 'name': 'Arrival Delays', 'type': 'histogram', 
                       'opacity': 0.75}, indices=0) 
            g.restyle({'x': [x2], 'name': 'Departure Delays', 'type': 'histogram', 
                       'opacity': 0.75}, indices=1)
            g.relayout({'barmode': 'overlay', 'xaxis': {'title': 'Delay in Minutes'},
                        'yaxis': {'title': 'Number of Delays'}})
            # print temp_df.head()
            
        else:
            # print "not using date"
            filter_list = [i and j for i,j in 
                           zip(df['carrier'] == 'DL', df['origin'] == origin.value)]
            temp_df = df[filter_list]
            x1 = temp_df['arr_delay']
            x2 = temp_df['dep_delay']
            g.restyle({'x': [x1], 'name': 'Arrival Delays', 'type': 'histogram', 
                       'opacity': 0.75}, indices=0) 
            g.restyle({'x': [x2], 'name': 'Departure Delays', 'type': 'histogram', 
                       'opacity': 0.75}, indices=1)
            g.relayout({'barmode': 'overlay'})
            # print temp_df.head()
    

origin.observe(response, names="value")
textbox.observe(response, names="value")
month.observe(response, names="value")
use_date.observe(response, names="value")

Time to try the app out!!

In [7]:
display(container)
container2 = widgets.HBox([origin, textbox])
display(container2)
display(g)

HBox(children=(Checkbox(value=True, description='Date: '), FloatSlider(value=1.0, continuous_update=False, description='Month:', max=12.0, min=1.0, step=1.0)))

HBox(children=(Dropdown(description='Origin Airport:', index=1, options=('EWR', 'LGA', 'JFK'), value='LGA'), Text(value='Delta', description='Airline:   ')))

GraphWidget()